In [1]:
import matplotlib.pyplot as plt
!pip install transformers datasets

In [2]:

from datasets import load_dataset

import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint

from transformers import AutoModelForSequenceClassification
import pandas as pd
import torch

from tools import download_dataset, load_dataset_into_to_dataframe, partition_dataset, IMDBDataset

## Load dataset

In [3]:
download_dataset()

data_frame = load_dataset_into_to_dataframe()
partition_dataset(data_frame)

100% | 80.23 MB 

100%|██████████| 50000/50000 [00:58<00:00, 852.02it/s] 


Class distribution:


In [4]:
df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")
df_test = pd.read_csv("test.csv")

## Tokenization

In [5]:
imdb_ds = load_dataset(
    "csv",
    data_files={
        "train": "train.csv",
        "validation": "val.csv",
        "test": "test.csv",
    },
)

print(imdb_ds)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 35000
    })
    validation: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 10000
    })
})


In [6]:
from transformers import AutoTokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print(f"Tokenizer input max length: {tokenizer.model_max_length}")
print(f"Tokenizer vocabulary size: {tokenizer.vocab_size}")

Tokenizer input max length: 512
Tokenizer vocabulary size: 30522


In [8]:
def tokenize_text(batch):
    return tokenizer(batch["text"], truncation=True, padding=True)

In [9]:
imdb_tokenized = imdb_ds.map(tokenize_text, batched=True, batch_size=None)

Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [10]:
del imdb_ds

In [11]:
imdb_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [12]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## Setup Dataloaders

In [13]:
train_ds = IMDBDataset(imdb_tokenized, partition_key="train")
val_ds = IMDBDataset(imdb_tokenized, partition_key="validation")
test_ds = IMDBDataset(imdb_tokenized, partition_key="test")

In [14]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=12,
    shuffle=True,
    num_workers=4,
)

val_loader = DataLoader(
    dataset=val_ds,
    batch_size=12,
    num_workers=4,
)
test_loader = DataLoader(
    dataset=test_ds,
    batch_size=12,
    num_workers=4,
)

## Initialize DistilBERT

In [15]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
for param in model.parameters():
    param.requires_grad = False

model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

## Finetuning DistilBERT

In [17]:
import torchmetrics

class LightningModel(L.LightningModule):
    def __init__(self, model, learning_rate=3e-5):
        super().__init__()
        self._learning_rate = learning_rate
        self._model = model

        self._val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)
        self._test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)

    def forward(self, input_ids, attention_mask, labels):
        return self._model(input_ids, attention_mask=attention_mask, labels=labels)

    def training_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"], labels=batch["label"])
        self.log("train_loss", outputs["loss"])
        return outputs["loss"]

    def validation_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"], labels=batch["label"])
        self.log("val_loss", outputs["loss"], prog_bar=True)
        logits = outputs["logits"]
        predicted_labels = torch.argmax(logits, 1)
        self._val_acc(predicted_labels, batch["label"])
        self.log("val_acc", self._val_acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"], labels=batch["label"])
        logits = outputs["logits"]
        predicted_labels = torch.argmax(logits, 1)
        self._test_acc(predicted_labels, batch["label"])
        self.log("accuracy", self._test_acc, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self._learning_rate)
        return optimizer


In [18]:
callbacks = [
    ModelCheckpoint(
        save_top_k=1, mode="max", monitor="val_acc"
    )
]
logger = CSVLogger(save_dir="logs/", name="lt_model")

## Finetune all layers (better do it on gpu)

In [19]:
all_layers_finetune_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
lt_model2 = LightningModel(all_layers_finetune_model)

trainer = L.Trainer(
    max_epochs=3,
    callbacks=callbacks,
    accelerator="auto",
    precision="32-true",
    devices="auto",
    logger=logger,
    log_every_n_steps=10,
)

trainer.fit(model=lt_model2,
            train_dataloaders=train_loader,
            val_dataloaders=val_loader,
            )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: logs/lt_model
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                                | Params
------------------------------------------------------------------
0 | _model    | DistilBertForSequenceClassification | 67.0 M
1 | _val_acc  | MulticlassAccuracy                  | 0     
2 | _test_acc | MulticlassAccuracy                  | 0     
------------------------------------------------------------------
67.0 M    Trainable params

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

D:\Programming Train\dl_lightning_ai_course\venv\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
D:\Programming Train\dl_lightning_ai_course\venv\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
trainer.test(lt_model2, dataloaders=train_loader, ckpt_path="best")

In [ ]:
trainer.test(lt_model2, dataloaders=val_loader, ckpt_path="best")

In [ ]:
trainer.test(lt_model2, dataloaders=test_loader, ckpt_path="best")

## Confidence Interval

In [ ]:
import scipy.stats

confidence = 0.95
z_value = scipy.stats.norm.ppf((1 + confidence) / 2.0)
print(z_value)

In [ ]:
n = len(test_ds)

In [ ]:
acc_test = 0.9312999844551086

In [ ]:
import numpy as np

SE = np.sqrt((acc_test * (1 + confidence) / 2.0))
ci_length = z_value * SE

In [ ]:
ci_lower = acc_test - ci_length
ci_upper = acc_test + ci_length

print(ci_lower, ci_upper)

In [ ]:
import matplotlib.pyplot as ptl

fig, ax = ptl.subplots(figsize=(7, 3))

ax.errorbar(0, acc_test, yerr=ci_length, fmt="o")

ax.set_ylim([0.9, 1.0])
ax.set_xticks(np.arange(1))
ax.set_xticklabels("Model 1")
ax.set_ylabel("Prediction accuracy")

plt.tight_layout()

plt.show()